In [2]:
from C_Classes import SUBMISSION_510K, SUBMISSION_DE_NOVO

In [6]:
Number = 'K232699' #Example of a 510(k) with a machine-readible PDF
Number = 'K991766' #Example of a 510(k) with a photo-copied and not machine-readible PDF
Number = 'DEN130045' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN130045
Number = 'DEN180044' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN180044
Number = 'DEN200030' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN200030

# # 510k class
# sub1 = SUBMISSION_510K(Number)
# sub1.get_510k_summary()
# sub1.get_510k_pdf()
# sub1.get_510k_IFU()
# sub1.summarize()

#De Novo class
sub2 = SUBMISSION_DE_NOVO(Number)
sub2.Submission_Type
sub2.Submission_Number
sub2.get_De_Novo_summary()
sub2.get_Reclassification_pdf()
sub2.get_FDA_Review_pdf()
sub2.get_De_Novo_IFU()
sub2.summarize()


Submission_Number: DEN180044 
Submission_Type: De Novo 510(k) 
URL: https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN180044 
Device_Classification_Name: electrocardiograph software for over-the-counter use 
De_Novo_Number: DEN180044 
Device_Name: ECG App 
Requester: Apple Incc/o biologics consulting group400 n washington st., suite 100alexandria,VA22314 
Contact: donna-bea  tillman 
Regulation_Number: 870.2345 
Classification_Product_Code: QDA 
Date_Received: 08/14/2018 
Decision_Date: 09/11/2018 
Decision: granted (DENG) 
Classification_Advisory_Committee: Cardiovascular 
Review_Advisory_Committee: Cardiovascular 
Reclassification_Order: Reclassification Order 
Reclassification_Order_URL: https://www.accessdata.fda.gov/cdrh_docs/pdf18/DEN180044.pdf 
FDA_Review: Decision Summary 
FDA_Review_URL: http://www.accessdata.fda.gov/cdrh_docs/reviews/DEN180044.pdf 
Type: Direct 
PDF_Reclassification_Downloaded: Yes 
PDF_Reclassification_Readable: Unknown 
PDF_Reclassif